### Test Problem

Same as in Lesson 1 Laplace with BCs:

$p = 0$ at $x = 0$

$\frac{\partial p}{\partial x} = 0$ at $x = L$

$p = 0$ at $y = 0$

$p = 0$ at $y = 0$

$p = \sin(\frac{\frac{3}{2} \pi x}{L})$ at $y = H$

In [2]:
import numpy
from matplotlib import pyplot, cm
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [3]:
from laplace_helper import p_analytical, plot_3D, L2_rel_error

In [4]:
nx = 128
ny = 128

L = 5
H = 5

x = numpy.linspace(0, L, nx)
y = numpy.linspace(0, H, ny)

dx = L / (nx - 1)
dy = H / (ny - 1)

p0 = numpy.zeros((ny, nx))

p0[-1, :] = numpy.sin(1.5 * numpy.pi * x / x[-1])

In [5]:
def laplace2d(p, l2_target):
    """Solves the Laplace equation using the Jacobi method with a 5-point stencil
    
    Parameters:
    ----------
    p : 2D array of float
        Initial potential distribution
    l2_target: float
        Stopping criterion
    
    Returns:
    -------
    p : 2D array of float
        Potential distribution after relaxation
    """
    
    l2norm = 1
    pn = numpy.empty_like(p)
    iterations = 0
    
    while l2norm > l2_target:
        pn = p.copy()
        p[1:-1, 1:-1] = .25 * (pn[1:-1, 2:] + pn[1:-1, :-2] + pn[2:, 1:-1] + pn[:-2, 1:-1])
        
        #Neumann BC along x = L
        p[1:-1, -1] = .25 * (2 * pn[1:-1, -2] + pn[2:, -1] + pn[:-2, -1])
        
        l2norm = numpy.sqrt(numpy.sum((p - pn) ** 2) / numpy.sum(pn ** 2))
        iterations += 1
        
    return p, iterations

In [6]:
l2_target = 1e-8
p, iterations = laplace2d(p0.copy(), l2_target)

print("Jacobi method took {} iterations at tolerance {}".format(iterations, l2_target))

Jacobi method took 19993 iterations at tolerance 1e-08


In [7]:
%%timeit
laplace2d(p0.copy(), l2_target)

1 loops, best of 3: 7.05 s per loop


In [8]:
pan = p_analytical(x, y)

In [9]:
L2_rel_error(p, pan)

6.1735513352884566e-05

## Gauss-Seidel

Single jacobi:

$$p_{i,j} ^{k+1} = \frac{1}{4} \left( p_{i,j-1} ^k + p_{i,j+1} ^k + p_{i-1,j} ^k + p_{i+1,j} ^k \right) $$

Gauss-Seidel is a simple tweak to this: use updated values of the solution as soon as they are available instead of waiting for the whole grid to be updated.

Gauss-Seidel:

$$p_{i,j} ^{k+1} = \frac{1}{4} \left( p_{i,j-1} ^ {k+1} + p_{i,j+1} ^k + p_{i-1,j} ^{k+1} + p_{i+1,j} ^k \right) $$

Can't use NumPy's array operations to evaluate the solution updates though.  GS requires using values immediately after they're updated, so we have to use nested for loops.

In [10]:
def laplace2d_gauss_seidel(p, nx, ny, l2_target):
    
    iterations = 0
    iter_diff = l2_target + 1 #init iter_diff to be larger than l2_target
    
    while iter_diff > l2_target:
        pn = p.copy()
        iter_diff = 0.0
        for j in range(1, ny - 1):
            for i in range(1, nx - 1):
                p[j, i] = .25 * (p[j, i - 1] + p[j, i + 1] + p[j - 1, i] + p[j + 1, i])
                iter_diff += (p[j, i] - pn[j, i]) ** 2
                
        #Neumann 2nd order BC
        for j in range(1, ny - 1):
            p[j, -1] = .25 * (2 * p[j, -2] + p[j + 1, -1] + p[j - 1, -1])
            
        iter_diff = numpy.sqrt(iter_diff / numpy.sum(pn ** 2))
        iterations += 1
        
    return p, iterations

In [11]:
import numba
from numba import jit

In [12]:
def fib_it(n):
    a = 1
    b = 1
    for i in range(n - 2):
        a, b = b, a + b
        
    return b

In [13]:
%%timeit
fib_it(500000)

1 loops, best of 3: 5.08 s per loop


In [14]:
@jit
def fib_it(n):
    a = 1
    b = 1
    for i in range(n - 2):
        a, b = b, a + b
        
    return b

In [15]:
%%timeit
fib_it(500000)

The slowest run took 223.11 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 601 µs per loop


In [16]:
%%timeit
fib_it(500000)

1000 loops, best of 3: 621 µs per loop


In [17]:
print(numpy.__version__)

1.10.1


Can't can't optimize everything.  When it can't, instead of failing it runs in regular python.  To avoid this you can add @jit(nopython=True) so that it fails instead of just running in regular python.  Fast or nothing

In [18]:
@jit(nopython=True)
def laplace2d_jacobi(p, pn, l2_target):
    """Solves the Laplace equation using the Jacobi method with a 5-point stencil
    
    Parameters:
    ----------
    p : 2D array of float
        Initial potential distribution
    pn : 2D array of float
        Allocated array for previous potential distribution
    l2_target : float
        Stopping criterion
    
    Returns:
    -------
    p : 2D array of float
        Potential distribution after relaxation
    """
    
    iterations = 0
    iter_diff = l2_target + 1 #init iter_diff to be larger than l2_target
    denominator = 0.0
    ny, nx = p.shape
    l2_diff = numpy.zeros(20000)
    
    while iter_diff > l2_target:
        for j in range(ny):
            for i in range(nx):
                pn[j, i] = p[j, i]
                
        iter_diff = 0.0
        denominator = 0.0
        
        for j in range(1, ny - 1):
            for i in range(1, nx - 1):
                p[j, i] = .25 * (pn[j, i - 1] + pn[j, i + 1] + pn[j - 1, i] + pn[j + 1, i])
                
                
        #Neumann 2nd order BC
        for j in range(1, ny - 1):
            p[j, -1] = .25 * (2 * pn[j, -2] + pn[j + 1, -1] + pn[j - 1, -1])
            
        for j in range(ny):
            for i in range(nx):
                iter_diff += (p[j, i] - pn[j, i]) ** 2
                denominator += (pn[j, i] * pn[j, i])
                
        
        iter_diff /= denominator
        iter_diff = iter_diff ** 0.5
        l2_diff[iterations] = iter_diff
        iterations += 1
        
    return p, iterations, l2_diff